# Smoking is bad

In [ ]:
# TODO: install all the necessary packages

In [ ]:
# TODO: load the data using the pathlib library to create the path to your data

In [ ]:
# TODO: what are the dimensions of the data frame?

In [ ]:
# TODO: use .describe() on your data frame: what do you notice about the string columns?

In [ ]:
# TODO: do not forget to check the null values

In [ ]:
# TODO: prepare your data frame so that you can work with

## Smoking analysis
Let's first check what your cousin did.

In [ ]:
# TODO: Reimplement the small code snippet described in the problem set instructions.

You should get the same results.

# Is smoking really good?
Now it is up to you to run the analysis further and check what could be underlying your cousings results.